In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Scikit Learn Imports
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
# Tensorflow Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense


In [2]:
# Import Data
data_import = pd.read_csv(".//house_predictions.csv")
print(data_import)

        price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no   

    hotwaterheating aircond

## Prepare Data

In [3]:
# Create pandas dataframe
house_info = pd.DataFrame(data_import)
house_info.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
# Check for nulls
house_info.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [5]:
# Drop unnecessary columns
house_df = house_info.drop(columns=["guestroom", "prefarea", "furnishingstatus"], axis=1)
house_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,basement,hotwaterheating,airconditioning,parking
0,13300000,7420,4,2,3,yes,no,no,yes,2
1,12250000,8960,4,4,4,yes,no,no,yes,3
2,12250000,9960,3,2,2,yes,yes,no,no,2
3,12215000,7500,4,2,2,yes,yes,no,yes,3
4,11410000,7420,4,1,2,yes,yes,no,yes,2


In [6]:
# Convert Yes/No to 1/0 for scaling
converted_house_df = house_df.copy()
converted_house_df[["mainroad", "basement", "hotwaterheating", "airconditioning"]] = converted_house_df[["mainroad", "basement", "hotwaterheating", "airconditioning"]].replace({"yes": 1, "no": 0})
converted_house_df.head()

C:\Users\Scarthicus\AppData\Local\Temp\ipykernel_26528\2219152172.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  converted_house_df[["mainroad", "basement", "hotwaterheating", "airconditioning"]] = converted_house_df[["mainroad", "basement", "hotwaterheating", "airconditioning"]].replace({"yes": 1, "no": 0})


,price,area,bedrooms,bathrooms,stories,mainroad,basement,hotwaterheating,airconditioning,parking
0,13300000,7420,4,2,3,1,0,0,1,2
1,12250000,8960,4,4,4,1,0,0,1,3
2,12250000,9960,3,2,2,1,1,0,0,2
3,12215000,7500,4,2,2,1,1,0,1,3
4,11410000,7420,4,1,2,1,1,0,1,2


In [7]:
# Split X and Y
y = converted_house_df["price"]

x = converted_house_df.copy()
x = x.drop(columns="price")

In [8]:
# Train Test Split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=4)

In [9]:
# Scale numeric data
scaler = StandardScaler().fit(x_train)
scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(x_test)

## Create Model

In [10]:
# Initial Model - Linear Regression
linear_regression_model = LinearRegression()

linear_regression_model.fit(x_train, y_train)
lr_model_predictions = linear_regression_model.predict(x_test)


In [11]:
# Score model
lr_model_r2_score = metrics.r2_score(y_test, lr_model_predictions)
lr_model_rmse = metrics.root_mean_squared_error(y_test, lr_model_predictions)
lr_model_mse = lr_model_rmse ** 2

print(f"R2 score: {lr_model_r2_score}  Mean Square Error: {lr_model_mse}  Root Mean Square Error: {lr_model_rmse}")

R2 score: 0.6101332275947464  Mean Square Error: 1252183947657.4966  Root Mean Square Error: 1119010.2535980162


In [12]:
# Alternate model - Random Forest
random_forest_model = RandomForestRegressor(n_estimators=1000, random_state=4)

random_forest_model.fit(scaled_x_train, y_train)
rf_model_predictions = random_forest_model.predict(scaled_x_test)

In [13]:
# Score model
rf_model_r2_score = metrics.r2_score(y_test, rf_model_predictions)
rf_model_rmse = metrics.root_mean_squared_error(y_test, rf_model_predictions)
rf_model_mse = rf_model_rmse ** 2

print(f"R2 score: {rf_model_r2_score}  Mean Square Error: {rf_model_mse}  Root Mean Square Error: {rf_model_rmse}")

R2 score: 0.5253645263629891  Mean Square Error: 1524446203533.5771  Root Mean Square Error: 1234684.6575274097


In [14]:
# Alternate model - Gradient Boosting Regression
gbr_model = GradientBoostingRegressor(n_estimators=1000, random_state=4)

gbr_model.fit(scaled_x_train, y_train)
gbr_model_predictions = gbr_model.predict(scaled_x_test)

In [15]:
# Score model
gbr_model_r2_score = metrics.r2_score(y_test, gbr_model_predictions)
gbr_model_rmse = metrics.root_mean_squared_error(y_test, gbr_model_predictions)
gbr_model_mse = gbr_model_rmse ** 2

print(f"R2 score: {gbr_model_r2_score}  Mean Square Error: {gbr_model_mse}  Root Mean Square Error: {gbr_model_rmse}")

R2 score: 0.43731582754579  Mean Square Error: 1807243238507.416  Root Mean Square Error: 1344337.4719568803


In [16]:
# Neural network model

# Build the neural network model
model = Sequential([
    Dense(512, activation='linear', input_shape=(scaled_x_train.shape[1],)),
    Dense(256,activation='linear'),
    Dense(256, activation='linear'),
    Dense(128, activation='linear'),
    Dense(64, activation='linear'),
    Dense(1, activation='linear')  # Single output neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(scaled_x_train, y_train, epochs=200, validation_split=0.2)

c:\Users\Scarthicus\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 27790896267264.0000 - val_loss: 25585231331328.0000
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 26663683358720.0000 - val_loss: 25582110769152.0000
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 26465624129536.0000 - val_loss: 25557938995200.0000
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 26893283753984.0000 - val_loss: 25407262818304.0000
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 26407358955520.0000 - val_loss: 24768107511808.0000
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 25093023465472.0000 - val_loss: 22671882452992.0000
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 21194906533888.0000 - val_loss: 18657304379392.0000
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17315881025536.0000 - val_loss: 14984623947776.0000
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 13857629143040.0000 - val_loss: 10433184399360.0000


In [19]:
# Evaluate the model
loss = model.evaluate(scaled_x_test, y_test)

# Make predictions
y_pred = model.predict(scaled_x_test)

# Calculate r2 value
r2 = metrics.r2_score(y_test, y_pred)

print(f"R-squared: {r2}")
print(f"Mean Squared Error on test set: {loss}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1439546343424.0000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
R-squared: 0.6150135688533596
Mean Squared Error on test set: 1236509130752.0


## Visualization

In [ ]:
# Visualization